In [ ]:
!pip install bokeh

In [26]:
import pandas as pd
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Slider, CategoricalColorMapper, CustomJS
from bokeh.layouts import column
from bokeh.palettes import Category10


data = pd.read_csv('gapminder.csv')

# population 5~55
data['pop_scaled'] = (data['pop'] / data['pop'].max()) * 50 + 5  

years = sorted(data['Year'].unique())
initial_year = years[0]

df_year = data[data['Year'] == initial_year]

# ColumnDataSource
source = ColumnDataSource(data=dict(
    x=df_year['Fertility'],
    y=df_year['lifeExp'],
    country=df_year['Country'],
    population=df_year['pop'],
    pop_size=df_year['pop_scaled'],
    region=df_year['Region'],
    year=df_year['Year']
))

# colors
regions = list(data['Region'].unique())
palette = Category10[10] if len(regions) <= 10 else Category10[10] * ((len(regions) // 10) + 1)
color_mapper = CategoricalColorMapper(factors=regions, palette=palette[:len(regions)])

#plot 
p = figure(title=f'Life Expectancy vs Fertility ({initial_year})',
           x_axis_label='Fertility (Children per Woman)',
           y_axis_label='Life Expectancy (Years)',
           width=900, height=600)

p.circle(x='x', y='y', size='pop_size', source=source,
         fill_alpha=0.6, line_color='black',
         color={'field': 'region', 'transform': color_mapper})

    
hover = HoverTool(tooltips=[
    ("Country", "@country"),
    ("Year", "@year"),
    ("Life Expectancy", "@y{0.0}"),
    ("Fertility", "@x{0.00}"),
    ("Population", "@population{0,0}"),
    ("Region", "@region")
])
p.add_tools(hover)

# year slider
slider = Slider(start=years[0], end=years[-1], value=initial_year, step=1, title="Year")

# full data 
full_source = ColumnDataSource(data=dict(
    Fertility=data['Fertility'],
    lifeExp=data['lifeExp'],
    Country=data['Country'],
    pop=data['pop'],
    pop_size=data['pop_scaled'],
    Region=data['Region'],
    Year=data['Year']
))

# JS Callback updated year
callback = CustomJS(args=dict(source=source, full_source=full_source, slider=slider), code="""
    var year = slider.value;
    var f_data = full_source.data;
    var new_x = [];
    var new_y = [];
    var new_country = [];
    var new_pop = [];
    var new_pop_size = [];
    var new_region = [];
    var new_year = [];
    for (var i=0; i<f_data['Year'].length; i++){
        if(f_data['Year'][i] == year){
            new_x.push(f_data['Fertility'][i]);
            new_y.push(f_data['lifeExp'][i]);
            new_country.push(f_data['Country'][i]);
            new_pop.push(f_data['pop'][i]);
            new_pop_size.push(f_data['pop_size'][i]);
            new_region.push(f_data['Region'][i]);
            new_year.push(f_data['Year'][i]);
        }
    }
    source.data = {
        x: new_x,
        y: new_y,
        country: new_country,
        population: new_pop,
        pop_size: new_pop_size,
        region: new_region,
        year: new_year
    };
    source.change.emit();
""")
slider.js_on_change('value', callback)


show(column(p, slider))


In [27]:
import pandas as pd
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Slider, CustomJS
from bokeh.layouts import column
from bokeh.palettes import Category10

# read data
data = pd.read_csv('gapminder.csv')
data['pop_scaled'] = (data['pop'] / data['pop'].max()) * 50 + 5

years = sorted(data['Year'].unique())
initial_year = years[0]

# Get all regions
regions = list(data['Region'].unique())
palette = Category10[10] if len(regions) <= 10 else Category10[10] * ((len(regions) // 10) + 1)

# Create for each region ColumnDataSource
sources = {}
for i, region in enumerate(regions):
    df_region = data[(data['Year']==initial_year) & (data['Region']==region)]
    sources[region] = ColumnDataSource(data=dict(
        x=df_region['Fertility'],
        y=df_region['lifeExp'],
        country=df_region['Country'],
        population=df_region['pop'],
        pop_size=df_region['pop_scaled'],
        year=df_region['Year']
    ))


p = figure(title=f'Life Expectancy vs Fertility ({initial_year})',
           x_axis_label='Fertility (Children per Woman)',
           y_axis_label='Life Expectancy (Years)',
           width=800, height=600)

# scatter
for i, region in enumerate(regions):
    p.scatter(x='x', y='y', size='pop_size', fill_alpha=0.6, line_color='black',
              fill_color=palette[i], legend_label=region, source=sources[region], name=region)

# Hover 
hover = HoverTool(tooltips=[
    ("Country", "@country"),
    ("Year", "@year"),
    ("Life Expectancy", "@y{0.0}"),
    ("Fertility", "@x{0.00}"),
    ("Population", "@population{0,0}"),
])
p.add_tools(hover)

p.legend.title = "Region"
p.legend.location = "top_right"
p.legend.click_policy = "hide"


slider = Slider(start=years[0], end=years[-1], value=initial_year, step=1, title="Year")

# JS callback updates the data of each region
callback = CustomJS(args=dict(sources=sources, slider=slider, full_data=data.to_dict('list')), code="""
    const year = slider.value;
    const data = full_data;
    const regions = Object.keys(sources);
    
    for (let i=0; i<regions.length; i++){
        const region = regions[i];
        const src = sources[region].data;
        const new_x = [], new_y = [], new_country = [], new_population = [], new_pop_size = [], new_year = [];
        for (let j=0; j<data['Year'].length; j++){
            if(data['Year'][j] === year && data['Region'][j] === region){
                new_x.push(data['Fertility'][j]);
                new_y.push(data['lifeExp'][j]);
                new_country.push(data['Country'][j]);
                new_population.push(data['pop'][j]);
                new_pop_size.push(data['pop_scaled'][j]);
                new_year.push(data['Year'][j]);
            }
        }
        src['x'] = new_x;
        src['y'] = new_y;
        src['country'] = new_country;
        src['population'] = new_population;
        src['pop_size'] = new_pop_size;
        src['year'] = new_year;
        sources[region].change.emit();
    }
""")
slider.js_on_change('value', callback)

# output HTML
output_file("life_expectancy_fertility_interactive.html")
show(column(p, slider))
